In [10]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from EC_data_processing_lib import get_header_length
from EC_data_processing_lib import analyse_file
from EC_data_processing_lib import get_decimal_separator
from pandas import DataFrame as df

fig = go.Figure()

#the following section uploads the data. You need to update the dataset array when you add or remove files. 
Co50Ni25Fe10Nx_dict = {
'Ru' : 0.85,  #measured resistance in ohms
'data' : 'Data_1/2022_01_24 Co50Ni25Fe25Nx elec1 set1_C05.mpt', #file path the slash needs to be '/' rather than the '\' which is automatic
'label' : 'Co<sub>50</sub>Ni<sub>25</sub>Fe<sub>10</sub>N<sub>X</sub>',
'cal' : 0.309, #calibration of the Hg/HgO electrode vs RHE converting to overpotential vs OER MUST BE UPDATED WHEN USING A DIFFERENT REF ELECTRODE
}

Co30Ni60Fe10Nx_dict = {
'Ru' : 0.85,  #measured resistance in ohms
'data' : 'Data_1/2022_01_25 Co30Ni60Fe10Nx 1mg  ref309 elect1 set_C05.mpt',
'label' : 'Co<sub>30</sub>Ni<sub>60</sub>Fe<sub>10</sub>N<sub>X</sub>',
'cal' : 0.309,
}

IrOx_dict = {
'Ru' : 0.85,  #measured resistance in ohms
'data' : 'Data_1/2022_01_26 IrOx 1mg  ref309 elect1_C05.mpt',
'label' : 'IrO<sub>X</sub>',
'cal' : 0.309,
}

dataset = [Co50Ni25Fe10Nx_dict,Co30Ni60Fe10Nx_dict, IrOx_dict]
for i in dataset:
    headerlength = get_header_length(i['data'])
    data = pd.read_csv(i['data'], encoding='ansi', sep="\t", decimal=get_decimal_separator(i['data']),  skiprows=range(headerlength))
    real = data['Re(Z)/Ohm']
    imaginary = data['-Im(Z)/Ohm']
    
      
   

    fig.add_trace(go.Scatter(
        x=real,
        y=imaginary,
        name= i['label']
    )
    
    
    
    )
fig.update_xaxes(range=[0, 2],
    nticks=10,
    ticks='outside',
    showgrid=False, gridwidth=1, gridcolor='grey',
    zeroline=False, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')

fig.update_yaxes(range=[0, 2],
    nticks=10,
    showgrid=False, gridwidth=1, gridcolor='grey',
    ticks='outside',
    zeroline=True, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')
fig.update_layout(
    #title="CV",
    xaxis_title="Time (min)",
    yaxis_title="Overpotential, η(mV) - iR<sub>u</sub>",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="dimgrey"
        
    )
)

fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    'paper_bgcolor': 'rgba(0, 0, 0, 0)', #sets the background to be transparent... might appear black in .jpeg but will be transparent in the .svg
})
fig.update_layout(
    width = 500,
    height = 500)
fig.show()
fig.write_image('PEIS.jpeg')
fig.write_image('PEIS.svg')
